In [1]:
import time
import random
import numpy as np
from sklearn.linear_model import LinearRegression
import collections

import altair as alt
import pandas as pd
import plotly.express as px
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 40, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 7, 5, 11
Xstd,Ystd = 1, 1

m1, k1 = 1, -1.0
m2, k2 = 1.5, 2.0
m3, k3 = 1, -1.5
m4, k4 = .75, 0.0

d1 = 1
d2 = 1
d3 = 1
d4 = 1

errorA = 4
errorB = 8
errorC = 2
errorX = 0.75
errorY = 0.6

def jitter(x, scale=1):
    return x + np.random.normal(scale=scale)

def dependent(x, m, c, error=1):
    return jitter(m*x + c, scale=error) # mx +c + error


def linear_positive_C_to_B(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        _, _, C = history[1 - d3]
        B_ = jitter(A*m1 + m3*C + k1 ,  errorB)
        _, B, _ = history[1 - d2]
        C_ = dependent(B, m2, k2, error=errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def linear_positive_A_to_C(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        B_ = dependent(A, m1, k1 ,  errorB)
        _, B, _ = history[1 - d2]
        A, _, _ = history[1 - d3]
        C_ = jitter(A*m3+ B*m2+ k2, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation # 0-1 , uniformly distributed
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def simulations_data(pathway, n=1000):
    random.seed(time.time())
    return np.array([pathway() for i in range(n)])


def regress(X,Y):
    model = LinearRegression()
    mXY = model.fit(X.reshape(-1,1), Y)
    r_sqr = mXY.score(X.reshape(-1,1), Y)
    residual = Y - model.predict(X.reshape(-1, 1))
    return mXY.intercept_, mXY.coef_[0], r_sqr, residual


def get_slope_intercept(model):
    return model._slopt, model._intercept

def compute_regresssion(ABC):
    A,B,C = ABC.transpose()
    RAB = regress(A, B)
    RBC = regress(B, C)
    RAC = regress(A, C)
    corrE = np.corrcoef(np.array([RAB[3], RBC[3]]))
    corrE_BA_C = np.corrcoef(np.array([np.square(RAB[3]), C]))


    #print(RAB[1]*RBC[1]-RAC[1]) ## better to look at distribution of this error..it should come with center as 0
    return {"kAB": RAB[0], "kBC":RBC[0], "kAC":RAC[0],
            "mAB":RAB[1], "mBC":RBC[1], "mAC":RAC[1],
            "r_sqrAB":RAB[2], "r_sqrBC":RBC[2], "r_sqrAC":RAC[2],
            "r_E":corrE[0,1],
            "r_E_BA_C":corrE_BA_C[0,1],
            "n":len(A)}

def compute_correlation(ABC):
    corr = np.corrcoef(ABC.transpose())
    rAB, rBC, rAC = corr[0,1], corr[1,2], corr[0,2]
    #print(rAB**2*rBC**2-rAC**2) ## better to look at distribution of this error..it should come with center as 0
                                ## or see correlation between these two quantities should be 1 and if we regress ,
                                ###       it should have slope 1
    return {"rAB":rAB, "rBC":rBC, "rAC":rAC}


def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        ABC = simulations_data(linear_positive_C_to_B, samplesize) # you can change pathway here
        r = compute_regresssion(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [2]:
def compute_confidence_interval(r, n):
    def boundary(zeta):
        return ((np.exp(2*zeta))-1)/((np.exp(2*zeta))+1)

    z = 0.5*np.log(((1-r)/(1+r)))
    zetal = z-1.96*np.sqrt(1/(n-3))
    rl = boundary(zetal)
    zetau = z+1.96*np.sqrt(1/(n-3))
    ru = boundary(zetau)
    return rl, ru

def confidence_status(L, U, v):
    l = v.where(v < L, "less")
    l.where(v > U, "more", inplace=True)
    w = l.where((v>= L) & (v <= U), "within")
    return w

def confidence_status_(r, n):
    L, U = compute_confidence_interval(r, n)
    return confidence_status(L, U, r)


def add_confidence_stats(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    L,U = compute_confidence_interval(d.rBC**2, d['n'])
    d['confidence_residual_corr'] = confidence_status(L, U, d.r_E)
    L,U = compute_confidence_interval(d.r_E, d['n'])#???
    d['confidence_corrected_bc_corr'] = confidence_status(L, U, d.r_E_BA_C**2)

def confidence_graphs(d):
    confidence = alt.Chart(d, title=f"Correlation confidence{(d['confidence']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_res_corr = alt.Chart(d, title=f"residual corr {(d['confidence_residual_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_residual_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_corrected_bc_corr = alt.Chart(d, title=f"corrected bc corr {(d['confidence_corrected_bc_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_corrected_bc_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    return alt.vconcat(confidence,confidence_res_corr, confidence_corrected_bc_corr)

add_confidence_stats(d)
confidence_graphs(d)

alt.VConcatChart(...)

In [8]:
def stats_graphs(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    slope_histogram = alt.Chart(d).mark_bar().encode(
        x=alt.X('mAB*mBC-mAC:Q', bin=True),
        y='count()').properties(title="slope diff histogram")

    bincount = 100
    ticks = 10

    correlation_graph = alt.Chart(d).mark_bar().encode(
        x=alt.X('rAB2*rBC2-rAC2:Q',bin=True,
                   axis=alt.Axis(
                    tickCount=ticks,
                    grid=False)),
        y='count()').properties(
            title="Correlation")
    residual_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X('r_E:Q', bin=True),
        y='count()').properties(title="Correlation of residuals")
    corrected_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X("r_E_BA_C2-rBC2:Q", bin=True),
        y='count()').properties(
        title="Corrected Correlation")

    return alt.vconcat(slope_histogram,correlation_graph,residual_correlation, corrected_correlation)

stats_graphs(d)

alt.VConcatChart(...)

In [4]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")

alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [5]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,...,n,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence,confidence_residual_corr,confidence_corrected_bc_corr
0,7.195207e+10,1.273620e+10,1.078066e+11,1.441255e+08,1.331082,2.098065e+08,0.104722,0.843770,0.105684,-0.024085,...,500,0.323608,0.918569,0.325091,-0.017323,-0.843765,-1.796363e+07,within,within,more
1,7.274673e+10,1.245464e+10,1.097815e+11,1.315875e+08,1.339071,1.782623e+08,0.075764,0.871247,0.067560,-0.002394,...,500,0.275253,0.933406,0.259922,-0.001550,-0.871222,-2.057288e+06,within,within,more
2,7.241421e+10,1.030639e+10,1.094000e+11,1.045767e+08,1.368704,1.436101e+08,0.057117,0.876617,0.050403,-0.000521,...,500,0.238993,0.936278,0.224507,-0.000333,-0.876308,-4.756077e+05,within,within,more
3,7.240606e+10,1.309327e+10,1.087912e+11,1.192662e+08,1.325961,1.654562e+08,0.067819,0.855283,0.063494,-0.007898,...,500,0.260421,0.924815,0.251980,-0.005489,-0.854780,-7.313856e+06,within,within,more
4,7.167073e+10,1.225787e+10,1.087683e+11,1.491770e+08,1.336856,1.814776e+08,0.098299,0.850075,0.069196,0.022187,...,500,0.313527,0.921995,0.263051,0.014366,-0.849986,1.795058e+07,within,within,more
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6.980750e+10,1.171802e+10,1.057659e+11,1.418551e+08,1.355201,2.057547e+08,0.100932,0.865218,0.100036,-0.018957,...,500,0.317698,0.930171,0.316285,-0.012708,-0.863195,-1.351253e+07,within,within,more
96,7.118030e+10,1.337451e+10,1.087890e+11,1.483835e+08,1.339122,1.964459e+08,0.089468,0.881728,0.077104,0.002909,...,500,0.299113,0.939004,0.277677,0.001782,-0.860460,2.257724e+06,within,within,more
97,7.264436e+10,1.215174e+10,1.097427e+11,1.236806e+08,1.344040,1.673749e+08,0.071479,0.863602,0.062581,-0.001284,...,500,0.267355,0.929302,0.250162,-0.000852,-0.859628,-1.143147e+06,within,within,more
98,7.373067e+10,1.527843e+10,1.115947e+11,1.035483e+08,1.303998,1.307592e+08,0.054860,0.858619,0.044174,0.004395,...,500,0.234223,0.926617,0.210175,0.002930,-0.852133,4.267609e+06,within,within,more


In [6]:
import pytest
pytest.approx(d.r_sqrAB) == d.rAB**2

True

In [7]:
compute_confidence_interval(d.rAC**2, d['n'])

(0    -0.191601
 1    -0.154337
 2    -0.137488
 3    -0.150349
 4    -0.155942
         ...   
 95   -0.186096
 96   -0.163690
 97   -0.149453
 98   -0.131357
 99   -0.194971
 Length: 100, dtype: float64,
 0    -0.018160
 1     0.020253
 2     0.037454
 3     0.024334
 4     0.018609
         ...   
 95   -0.012453
 96    0.010660
 97    0.025250
 98    0.043688
 99   -0.021660
 Length: 100, dtype: float64)